In [2]:
# import sys
import time
import random
import carla
import ipdb
import pickle

In [3]:
traffic_light_dic  = {} 

In [4]:
# Get client to interact with CARLA server
client = carla.Client('127.0.0.1', 2000)

# Get current CARLA world
world = client.get_world()
Map = world.get_map()

In [39]:
def draw_waypoint(road_id, lane_id):
    waypoint_list = world.get_map().generate_waypoints(3)
    for waypoint in waypoint_list:
            # get a tl object
            rid = waypoint.road_id
            lid = waypoint.lane_id
            
            if rid == road_id and lid == lane_id:
                loc = waypoint.transform.location
                loc.x += 0.5
                world.debug.draw_string(loc, str(rid)+','+str(lid), draw_shadow=True,
                                        color=carla.Color(r=255, g=255, b=0), life_time=200)

In [42]:
traffic_light_dic  = {}
del_list = {}

In [47]:
all_actors = world.get_actors()
all_vehicles = all_actors.filter("vehicle*")
all_traffic_lights = all_actors.filter("traffic_light*")


while True:
    for vehicle in all_vehicles:
        # check if vehicle is at a traffic light, only then its traffic lights are valid
        # if vehicle.is_at_traffic_light() == True:
        tl = vehicle.get_traffic_light()
        state = vehicle.get_traffic_light_state()
        
        if (tl is not None) and (str(state) == 'Red'):
            if tl.get_location().distance(vehicle.get_location()) > 50:
                continue
            
            # check if traffic light is accessible
            # if traffic light available, get road_id
            tl_x_location = tl.get_location().x
            tl_y_location = tl.get_location().y

            veh_location = vehicle.get_location()
            waypoint = Map.get_waypoint(veh_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            road_id = waypoint.road_id
            lane_id = waypoint.lane_id
            section_id = waypoint.section_id

            if (road_id, lane_id) in traffic_light_dic and (road_id, lane_id) not in del_list:
                if traffic_light_dic[(road_id, lane_id)] != tl_x_location + tl_y_location:
                    draw_waypoint(road_id, lane_id)
                    print("we have a problem------------------------------------")
                    print(road_id, lane_id, tl_x_location, tl_y_location, traffic_light_dic[(road_id, lane_id)], tl_x_location + tl_y_location)
                    # deleting the problem creators. CHECK LOGIC.
                    del_list[(road_id, lane_id)] = 1
            
            if road_id < 100:
                if (road_id, lane_id) not in traffic_light_dic and (road_id, lane_id) not in del_list:
                    traffic_light_dic[(road_id, lane_id)] = tl_x_location + tl_y_location
                    print(road_id, lane_id, tl_x_location, tl_y_location, tl_x_location + tl_y_location, "traffic light added ", len(traffic_light_dic))

8 -2 -139.34999084472656 -100.8499984741211 -240.19998931884766 traffic light added  115
we have a problem------------------------------------
8 1 -140.39999389648438 -78.0999984741211 -192.09999084472656 -218.49999237060547
we have a problem------------------------------------
23 2 -139.8000030517578 77.8499984741211 -11.199996948242188 -61.95000457763672
44 -1 -140.39999389648438 -78.0999984741211 -218.49999237060547 traffic light added  116
we have a problem------------------------------------
3 1 -201.5500030517578 12.449999809265137 -166.89999103546143 -189.10000324249268
we have a problem------------------------------------
43 -2 -139.8000030517578 77.8499984741211 -188.94999980926514 -61.95000457763672
39 1 40.25 -11.09999942779541 29.15000057220459 traffic light added  117
39 2 40.89999771118164 10.5 51.39999771118164 traffic light added  118


KeyboardInterrupt: 

In [48]:
for (road_id, lane_id) in del_list.keys():
    draw_waypoint(road_id, lane_id)
del_list

{(23, 1): 1,
 (39, -1): 1,
 (39, -2): 1,
 (8, 1): 1,
 (23, 2): 1,
 (3, 1): 1,
 (43, -2): 1}

In [61]:
for key, value in traffic_light_dic.items():
    if int(value) == 29:
        print(key)

(26, 2)
(26, 1)
(39, 1)


In [63]:
del traffic_light_dic[(42,-1)]
del traffic_light_dic[(23,-1)]
del traffic_light_dic[(8,-2)]
del traffic_light_dic[(20,-1)]
del traffic_light_dic[(39,2)]
del traffic_light_dic[(39,1)]

In [64]:
import pickle
f = open("traffic_lights/town05_traffic_light_x_V2.pkl","wb")
pickle.dump(traffic_light_dic, f)
f.close()

In [28]:
tl_dict = pickle.load(open( "town05_traffic_light_x.pkl", "rb" ) )

In [42]:
%%timeit
dic = {}
for tl in all_traffic_lights:
    x = tl.get_location().x
    for key, value in tl_dict.items():
        if x == value:
            dic[key] = tl.id

265 µs ± 792 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
loc = tl.get_location()

In [52]:
loc.distance(loc)

0.0

In [107]:
vehicles_list = list(world.get_actors().filter("vehicle*"))
print(vehicles_list)

[<carla.libcarla.Vehicle object at 0x7fc912185450>, <carla.libcarla.Vehicle object at 0x7fc911dde090>, <carla.libcarla.Vehicle object at 0x7fc911dde0f0>, <carla.libcarla.Vehicle object at 0x7fc911dde150>, <carla.libcarla.Vehicle object at 0x7fc911dde390>, <carla.libcarla.Vehicle object at 0x7fc911dde210>, <carla.libcarla.Vehicle object at 0x7fc911dde2d0>, <carla.libcarla.Vehicle object at 0x7fc911dde3f0>, <carla.libcarla.Vehicle object at 0x7fc911dde330>, <carla.libcarla.Vehicle object at 0x7fc911dde450>]


In [78]:
Map = world.get_map()

In [85]:
get_road_id = lambda veh : Map.get_waypoint(veh.get_location()).road_id

In [110]:
print(vehicles_list[0].get_transform().location.x)

-474.14739990234375


In [100]:
a = [3,4,5,6,7,8,9,1,2,0]

In [103]:
vehicles_list.sort(key=a)
print([x.id for x in vehicles_list])

TypeError: 'list' object is not callable

In [29]:
# making a dict of traffic_light_x: traffic_light_actor_id
tl_actor_id_dict = {}
a ={}
for tl in all_traffic_lights:
    if tl
    tl_actor_id_dict[tl.get_location().x] = tl.id
    a[tl.id] = tl.get_location().x
print(len(tl_actor_id_dict))

46


Facts gathered:
1. Each traffic light actor HAS a unique x location
2. Each (road_id, lane_id) doesnt have to point to a unique traffic light
3. Section ids for all gathered lanes were zero
4. 